In [1]:
%pip install SpeechRecognition 
%pip install pydub
%pip install openai-whisper
%pip install ffmpeg

  Using cached SpeechRecognition-3.14.0-py3-none-any.whl.metadata (31 kB)
Using cached SpeechRecognition-3.14.0-py3-none-any.whl (32.9 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-whisper-20240930.tar.gz (800 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached more_itertools-10.6.0-py3-none-any.whl.metadata (37 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached more_itertools-10.6.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()

# Specify the path to your MP3 file
mp3_file = "audio2.mp3"

# Convert MP3 to WAV
from pydub import AudioSegment
audio = AudioSegment.from_mp3(mp3_file)
wav_file = "temp_audio.wav"
audio.export(wav_file, format="wav")

# Recognize the speech in the audio file
import whisper

def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    result = model.transcribe(audio_file)
    return result["text"]

transcript = transcribe_audio(wav_file)
print(transcript)



/Users/neelkolhe/Downloads/theoffice/.venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/neelkolhe/Downloads/theoffice/.venv/lib/python3.9/site-packages/pydub/utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe'